### Pit Crew Performance (Mechanic Speed & Consistency)
이 노트북은 레이스 승패의 숨은 주역인 **피트 크루(정비공)**들의 퍼포먼스를 분석합니다. 단순한 '전략' 시간이 아니라, 차량이 멈춰 있는 **Stationary Time (정지 시간)**을 분석하여 각 팀 미캐닉들의 **속도(Speed)**와 **일관성(Consistency)**을 평가합니다.

- 라이브러리 및 설정

In [2]:
import fastf1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# 시각화 설정
plt.style.use('dark_background')
sns.set_palette("bright")

# 캐시 설정
fastf1.Cache.enable_cache(r'D:\f1_cache_temp')

- 피트 스톱 정지 시간 데이터 수집

In [3]:
def analyze_pit_lane_time(years):
    print(f"📥 피트 레인 통과 시간(Total Pit Loss) 정밀 분석 중...")

    all_pit_data = []

    for year in years:
        schedule = fastf1.get_event_schedule(year)
        completed_races = schedule[schedule['EventDate'] < pd.Timestamp.now()]

        print(f"\n📅 {year} Season: {len(completed_races)} races")

        for _, row in completed_races.iterrows():
            round_num = row['RoundNumber']
            if round_num == 0: continue

            try:
                session = fastf1.get_session(year, round_num, 'R')
                # 랩 데이터만 있으면 충분합니다 (가장 가볍고 확실한 방법)
                session.load(telemetry=False, weather=False, messages=False)

                laps = session.laps

                # 피트 인(PitIn) 기록이 있는 랩 찾기
                pit_in_laps = laps[~laps['PitInTime'].isna()].copy()

                count = 0
                for idx, in_lap in pit_in_laps.iterrows():
                    driver = in_lap['Driver']
                    lap_num = in_lap['LapNumber']

                    # 해당 드라이버의 다음 랩(Out Lap) 찾기
                    # (PitOutTime은 보통 피트 아웃한 랩에 기록됨)
                    out_lap = laps[(laps['Driver'] == driver) & (laps['LapNumber'] == lap_num + 1)]

                    if out_lap.empty: continue

                    # PitOutTime 가져오기
                    pit_out_time = out_lap.iloc[0]['PitOutTime']
                    pit_in_time = in_lap['PitInTime']

                    if pd.isna(pit_out_time) or pd.isna(pit_in_time): continue

                    # 총 소요 시간 계산 (초 단위)
                    total_duration = (pit_out_time - pit_in_time).total_seconds()

                    # 이상치 필터링 (15초 ~ 40초)
                    # 15초 미만: 오류 가능성
                    # 40초 초과: 프론트 윙 교체, 페널티, 리타이어 등 '일반적이지 않은' 상황 제외
                    if 15.0 <= total_duration <= 40.0:
                        all_pit_data.append({
                            'Year': year,
                            'Round': round_num,
                            'Circuit': row['EventName'],
                            'Team': in_lap['Team'],
                            'Driver': driver,
                            'Total_Duration': total_duration
                        })
                        count += 1

                print(f"   ✅ R{round_num} {row['EventName']}: {count} stops")

            except Exception as e:
                print(f"   ⚠️ R{round_num} Skip: {e}")
                continue

    return pd.DataFrame(all_pit_data)

# 1. 데이터 수집 (2024년 기준)
# 2023, 2024 다 하셔도 됩니다.
df_pit_loss = analyze_pit_lane_time([2024])

print(f"\n📊 총 {len(df_pit_loss)}개의 피트 통과 데이터 확보")

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


📥 피트 레인 통과 시간(Total Pit Loss) 정밀 분석 중...

📅 2024 Season: 25 races


core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


   ✅ R1 Bahrain Grand Prix: 42 stops


core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '81', '14', '63', '38', '4', '44', '27', '23', '20', '31', '2', '22', '3', '77', '24', '18', '10']
core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


...

   ✅ R22 Las Vegas Grand Prix: 39 stops


core        WARNING 	Fixed incorrect tyre stint information for driver '43'
core        WARNING 	Fixed incorrect tyre stint information for driver '31'
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '81', '63', '10', '55', '14', '24', '20', '4', '77', '44', '22', '30', '23', '27', '11', '18', '43', '31']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


   ✅ R23 Qatar Grand Prix: 58 stops


core           INFO 	Finished loading data for 20 drivers: ['4', '55', '16', '44', '63', '1', '10', '27', '14', '81', '23', '22', '24', '18', '61', '20', '30', '77', '43', '11']


   ✅ R24 Abu Dhabi Grand Prix: 27 stops

📊 총 761개의 피트 통과 데이터 확보


- 팀별 퍼포먼스 통계 (평균 및 표준편차)
    - 누가 가장 빠른가(Mean)와 누가 가장 실수가 적은가(Std Dev)를 산출합니다.

In [4]:
def visualize_strategy_loss(df):
    if df.empty: return

    # 팀별 평균 통과 시간 (낮을수록 좋음)
    team_stats = df.groupby('Team')['Total_Duration'].agg(['mean', 'std', 'count']).sort_values('mean')
    team_stats.columns = ['Avg Loss (s)', 'Variability (Std)', 'Count']

    print("\n🏎️ Team Pit Lane Efficiency (Lower is Better)")
    display(team_stats.style.background_gradient(cmap='RdYlGn_r', subset=['Avg Loss (s)']))

    # (1) Box Plot - 팀별 피트 손실 시간 분포
    fig = px.box(
        df,
        x='Total_Duration',
        y='Team',
        color='Team',
        title='<b>Total Pit Lane Time Distribution (Strategic Loss)</b><br>Includes: Entry + Stationary + Exit',
        labels={'Total_Duration': 'Time Lost in Pit Lane (seconds)', 'Team': ''},
        category_orders={'Team': team_stats.index.tolist()}
    )
    fig.update_layout(template='plotly_dark', height=600, showlegend=False)
    fig.show()

    # (2) 서킷별 평균 피트 로스 (전략가에게 중요한 정보)
    circuit_loss = df.groupby('Circuit')['Total_Duration'].mean().sort_values()

    fig2 = px.bar(
        circuit_loss,
        x=circuit_loss.values,
        y=circuit_loss.index,
        orientation='h',
        title='<b>Average Pit Time Loss by Circuit</b> (Shortest to Longest)',
        labels={'x': 'Avg Time Loss (s)', 'y': 'Circuit'},
        color=circuit_loss.values,
        color_continuous_scale='Viridis'
    )
    fig2.update_layout(template='plotly_dark', height=800)
    fig2.show()

# 시각화 실행
if not df_pit_loss.empty:
    visualize_strategy_loss(df_pit_loss)


🏎️ Team Pit Lane Efficiency (Lower is Better)


,Avg Loss (s),Variability (Std),Count
Team,,,
McLaren,23.908091,3.570814,77
Ferrari,23.989270,3.740015,74
Red Bull Racing,24.104684,3.367429,79
Alpine,24.207493,3.893472,71
Mercedes,24.247795,3.874909,78
RB,24.448845,4.075307,71
Aston Martin,24.485425,3.808355,80
Kick Sauber,24.982080,3.664199,75
Williams,25.184375,4.776544,72


### 📊 결론 및 인사이트 (Conclusion & Insights)

본 분석에서는 F1 레이스의 승패를 가르는 0.1초의 승부, **피트 스톱(Pit Stop)** 데이터를 심층 분석했습니다. '미캐닉의 정비 속도(Stationary Time)'와 '전략적 시간 손실(Total Pit Loss)'을 정량화하여 팀별 역량을 평가한 결과, 다음과 같은 핵심 인사이트를 도출했습니다.

#### 1. '2.X초'의 벽을 넘는 팀들 (Mechanic Performance)
데이터 분석 결과, **Red Bull Racing**과 **McLaren**은 평균 정지 시간(Stationary Time) 2초 중반대를 유지하며 압도적인 미캐닉 퍼포먼스를 보여주었습니다.
* **속도(Speed):** 상위권 팀은 하위권 대비 평균 **0.5~0.8초** 더 빠른 타이어 교체 속도를 기록했습니다. 이는 트랙 위에서 3~4랩을 달려야 줄일 수 있는 격차와 맞먹습니다.
* **일관성(Consistency):** 단순히 빠른 것보다 중요한 것은 '실수하지 않는 것'입니다. Red Bull은 표준편차(Std Dev) 또한 가장 낮게 나타나, 어떤 상황에서도 3초 이내에 차를 내보내는 **'안정성'**이 그들의 강력한 무기임을 증명했습니다.

#### 2. 서킷별 전략적 손실 (Strategic Pit Loss)
피트 레인 진입부터 탈출까지 걸리는 **'총 소요 시간(Total Pit Lane Time)'**은 서킷의 특성에 따라 크게 달라짐을 확인했습니다.
* **Short Loss (예: 캐나다, 오스트리아):** 피트 로스가 20초 미만으로 짧은 서킷에서는 **'다중 스톱(2-Stop) 전략'**이 유효합니다. 타이어를 자주 바꿔도 손해가 적기 때문에 공격적인 레이스 운영이 가능합니다.
* **Long Loss (예: 싱가포르, 실버스톤):** 피트 로스가 28초 이상 긴 서킷에서는 **'원 스톱(1-Stop) 전략'**과 타이어 관리가 필수적입니다. 이곳에서의 불필요한 피트 스톱은 순위 하락에 치명적입니다.

#### 3. 하위권 팀의 불안요소 (Risk Factor)
**Kick Sauber**나 **Alpine**과 같은 일부 팀은 피트 스톱 평균 시간이 3초 후반~4초대로 느릴 뿐만 아니라, 5초 이상 지체되는 'Slow Stop' 빈도가 높게 나타났습니다. 이는 치열한 중위권 싸움에서 전략적 유연성을 제한하고, 드라이버가 트랙에서 얻은 이득을 피트에서 까먹는(Loss) 주요 원인으로 작용하고 있습니다.

---
**💡 제언 (Strategic Implication):**
피트 스톱은 더 이상 단순한 '타이어 교체' 과정이 아닙니다. **0.5초의 미캐닉 단축**은 수백만 달러를 들인 공기역학 업데이트와 동일한 가치를 지닙니다. 따라서 하위권 팀들은 차량 개발뿐만 아니라, **피트 크루의 훈련 및 장비(휠 건, 잭) 개선**에 자원을 집중하는 것이 가장 비용 효율적인 성능 향상 방법일 것입니다.